In [1]:
import requests

auth_url = "https://signin.tradestation.com/authorize"

public_key = "r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4"
private_key = "hFBk8xgV_UGJEUFnxVW-AFz6YToqZwdvM-48x5wLUQhzKiR99r2w780hL0giBfvd"

params = {"response_type": "code", "client_id": public_key, "redirect_uri": "http://localhost", "audience": "https://api.tradestation.com", "state": "STATE", 
        "scope": "openid offline_access profile MarketData ReadAccount Trade Crypto Matrix OptionSpreads"}

response = requests.get(auth_url, params=params)
response.history
for resp in response.history:
        print(resp.status_code, resp.url)

302 https://signin.tradestation.com/authorize?response_type=code&client_id=r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4&redirect_uri=http%3A%2F%2Flocalhost&audience=https%3A%2F%2Fapi.tradestation.com&state=STATE&scope=openid+offline_access+profile+MarketData+ReadAccount+Trade+Crypto+Matrix+OptionSpreads


In [ ]:
import requests

token_url = "https://signin.tradestation.com/oauth/token"

authorization_code = "xxMZvFaqtX95pCIj"

headers = {"content-type": "application/x-www-form-urlencoded"}
data = {"grant_type": "authorization_code", "client_id": public_key, "client_secret": private_key, "code": authorization_code, "redirect_uri": "http://localhost"}

r = requests.post(token_url, headers=headers, data=data).json()
print(r)

access_token = r["access_token"]
refresh_token = r["refresh_token"]
id_token = r["id_token"]

In [2]:
import requests
refresh_token = "lzijZJe0VUnjFEzq6dqWK-Q3rqhzyoq7kkrDkqVoJvoYn"
def refresh():
    token_url = "https://signin.tradestation.com/oauth/token"

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "refresh_token", "client_id": public_key, "client_secret": private_key, "refresh_token": refresh_token}

    r = requests.post(token_url, headers=headers, data=data).json()

    access_token = r["access_token"]
    return access_token

In [ ]:
import requests
import pandas as pd
import json
import csv

url = "https://api.tradestation.com/v3/marketdata/barcharts/TSLA"

headers = {"Authorization": f"Bearer {access_token}"}

params = {"unit": "Minute", "firstdate": "2016-12-05T21:00:00Z", "lastdate": "2016-12-06T21:00:00Z"}

response = requests.request("GET", url, headers=headers, stream=True, params=params)
print(response)
if response.status_code == 401:
    access_token = refresh()
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.request("GET", url, headers=headers, stream=True, params=params)
    print(response)
response = response.json()
    
fieldnames = list(response["Bars"][0].keys())
with open("tradestation_data/tsla_2016_12_05_2016_12_06.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in response["Bars"]:
        writer.writerow(row)

In [ ]:
def stream_lvl1_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/quotes/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue   

            for line in response.iter_lines():
                if line:
                    row = json.loads(line)
                    if "Error" in row:
                        print(f"{symbol} {response} {response.text}")
                        continue
                    if "Heartbeat" in row:
                        continue
                    print(row)
        except Exception as e:
            print(f"{symbol} {e}")
            continue

In [ ]:
import os
import csv
import json
import pandas as pd
import datetime
import time

def stream_lvl2_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/marketdepth/aggregates/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue            
            
            for line in response.iter_lines():
                if line:
                    bids_and_asks = json.loads(line)
                    if "Heartbeat" in bids_and_asks:
                        continue
                    print(bids_and_asks)
        except Exception as e:
            print(f"{symbol} {e}")
            break
                    
symbols = "TSLA"
stream_lvl2_quotes(symbols)

In [5]:
import requests
account_id = "11655345"
def buy_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
buy_share("TSLA")

{"Orders":[{"Message":"Sent order: Buy 1 TSLA @ Market","OrderID":"967793406"}]}



In [11]:
import requests
def sell_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "SELL",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
sell_share("TSLA")

{"Orders":[{"Message":"Sent order: Sell 1 TSLA @ Market","OrderID":"967802392"}]}



In [13]:
import requests
def check_status():
    access_token = refresh()
    url = "https://api.tradestation.com/v3/brokerage/accounts/11655345/orders"

    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request("GET", url, headers=headers)

    print(response.text)
check_status()

{"Orders":[{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:10Z","Currency":"USD","Duration":"DAY","FilledPrice":"0","Legs":[{"QuantityOrdered":"1","ExecQuantity":"0","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK"}],"OrderID":"967805697","OpenedDateTime":"2023-06-07T19:53:10Z","OrderType":"Market","PriceUsedForBuyingPower":"224.98","RejectReason":"EC501: Day trading margin rules, this trade can possibly designate you as a pattern day trader, please call the Trade Desk at 800-871-3563 for more information.","Routing":"Intelligent","Status":"REJ","StatusDescription":"Rejected","UnbundledRouteFee":"0"},{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:00Z","Currency":"USD","Duration":"DAY","FilledPrice":"224.9699","Legs":[{"OpenOrClose":"Open","QuantityOrdered":"1","ExecQuantity":"1","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK","ExecutionPrice":"224.9699"}],"OrderID":

In [44]:
import datetime
import requests
account_id = "11655345"
def store_price(symbol):
    url = "https://api.tradestation.com/v3/orderexecution/orderconfirm"
    access_token = refresh()
    payload = {
        "AccountID": account_id,
        "Symbol": symbol,
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {"content-type": "application/json", "Authorization": f"Bearer {access_token}"}

    order_time = datetime.datetime.utcnow()
    time = order_time.replace(second=order_time.second - order_time.second%10).strftime("%Y-%m-%dT%H:%M:%SZ")
    response = requests.request("POST", url, json=payload, headers=headers).json()
    print(response)
    print(response["Confirmations"][0])
    buy_price = response["Confirmations"][0]["EstimatedPrice"]
    print(buy_price)
store_price("TSLA")

{'Confirmations': [{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}]}
{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}
226.53


In [ ]:
import requests

access_token = refresh()
account_id = "11655345"
url = f"https://api.tradestation.com/v3/brokerage/stream/accounts/{account_id}/orders"

headers = {"Authorization": f"Bearer {access_token}"}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)

In [2]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()

In [ ]:
import numpy as np
import pandas as pd
def eval_signal(symbol, dates, hour, price_dif_thresholds, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    df_dict = {}
    df_len = 0
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:05:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        cols = ["Time", "Last", "TotalSize", "Dif"]
        for k in range(1, 25):
            df_1[f"{k}0sec Dif"] = df_1["Last"].shift(-k) - df_1["Last"]
            df_1[f"{k}0sec Sign"] = df_1[f"{k}0sec Dif"] >= 0
            cols.append(f"{k}0sec Dif")
            cols.append(f"{k}0sec Sign")

        end = f'{date}T{hour+1}:00:00Z'
        query_2 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_2)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df = df.rename(columns={"CurTime": "Time"})
        df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
        df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
        df.loc[df["Side"] == "Ask", "TotalSize"] *= -1
        df = df[cols]
        df_dict[date] = df

    arr = np.zeros((300, 49))
    arr[:, 0] = price_dif_thresholds
    for i, price_dif_threshold in enumerate(price_dif_thresholds):
        temp_arr = np.zeros((0, 25))
        sign_count = np.zeros(24)
        total_count = 0
        for j, date in enumerate(dates):
            df = df_dict[date].copy()
            df = df.loc[df["Dif"] >= price_dif_threshold]
            if len(df) == 0:
                continue
            df["VolumeSum"] = df["TotalSize"] * df["Dif"]
            cols = ["Time", "Last", "VolumeSum"]
            cols_dict = {"VolumeSum": "sum", "Last": "first"}
            for k in range(1,25):
                cols.append(f"{k}0sec Dif")
                cols.append(f"{k}0sec Sign")
                cols_dict[f"{k}0sec Dif"] = "first"
                cols_dict[f"{k}0sec Sign"] = "first"
            df = df[cols]
            df = df.groupby("Time").agg(cols_dict).reset_index()
            df["Sign"] = df["VolumeSum"] >=0
            cols = ["VolumeSum"]
            for k in range(1,25):
                df["Equal"] = df["Sign"] == df[f"{k}0sec Sign"]
                sign_count[k-1] += df["Equal"].values.sum()
                cols.append(f"{k}0sec Dif")
            total_count += len(df)
            temp_arr = np.concatenate((temp_arr, df[cols].to_numpy()))
        if total_count == 0:
            continue
        temp_df = pd.DataFrame(temp_arr)
        corr_df = temp_df.corr()
        arr[i, 1:25] = corr_df.to_numpy()[0, 1:]
        arr[i, 25:49] = sign_count / total_count

    cols = ["Threshold"]
    for k in range(1,25):
        cols.append(f"{k}0sec Corr")
    for k in range(1,25):
        cols.append(f"{k}0sec Sign %")
    out_df = pd.DataFrame(arr, columns=cols)
    out_df.to_csv(f"tradestation_data/TSLA_{hour}_eval.csv")
    corrs = arr[:, 1:25]
    corr_threshold, corr_sec_dif = np.unravel_index(np.nanargmax(corrs), corrs.shape)
    signs = arr[:, 25:49]
    sign_threshold, sign_sec_dif = np.unravel_index(np.nanargmax(signs), signs.shape)
    return np.nanmax(corrs), price_dif_thresholds[corr_threshold], corr_sec_dif, np.nanmax(signs), price_dif_thresholds[sign_threshold], sign_sec_dif

symbol = "TSLA"
#price_dif_threshold = .03
price_dif_thresholds = (-np.arange(300) / 1000) + .2
size_threshold = 1
# for size_threshold in range(1, 100, 1):
hour = 13
days = ["13", "14", "15", "16"]
dates = [f"2023-06-{day}" for day in days]
print(f"{hour}: {eval_signal(symbol, dates, hour, price_dif_thresholds, size_threshold)}")
days = ["08", "09", "12", "13", "14", "15", "16"]
dates = [f"2023-06-{day}" for day in days]
for hour in range(14, 20):
    print(f"{hour}: {eval_signal(symbol, dates, hour, price_dif_thresholds, size_threshold)}")


In [5]:
import numpy as np
import pandas as pd
def calc_gain_long(df, buy_threshold, sell_threshold, size_threshold):
    last_sell = df.iloc[len(df) - 1, df.columns.get_loc("SELL")]
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] >= buy_threshold))
                | ((df["Side"] == "Ask") & (df["Dif"] <= sell_threshold))]
    df.loc[df["Side"] == "Bid", "Multiplier"] = df.loc[df["Side"] == "Bid", "Dif"] - buy_threshold
    df.loc[df["Side"] == "Ask", "Multiplier"] = df.loc[df["Side"] == "Ask", "Dif"] - sell_threshold
    df["VolumeSum"] = df["TotalSize"] * df["Multiplier"]
    df = df[["Time", "VolumeSum", "BUY", "SELL"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first"})
    df = df[(df["VolumeSum"] > size_threshold) | (df["VolumeSum"] < -size_threshold)].reset_index()
    df["Long"] = df["VolumeSum"] > size_threshold
    df["Short"] = df["VolumeSum"] < -size_threshold
    if len(df) < 2:
        return 0, None

    df["Buy Action"] = df["Long"] & df["Short"].shift()
    if df.iloc[0, df.columns.get_loc("Long")]:
        df.iloc[0, df.columns.get_loc("Buy Action")] = True
    df["Sell Action"] = df["Short"] & df["Long"].shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "BUY"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "SELL"])

    if df.iloc[len(df) - 1, df.columns.get_loc("Long")]:
        df.loc[len(df)] = [0, 0, 0, 0, 0, 0, 0, 0, last_sell]
    return df["Gain"].sum(), df

def calc_gain_short(df, buy_to_cover_threshold, sell_short_threshold, size_threshold):
    last_buy_to_cover = df.iloc[len(df) - 1, df.columns.get_loc("BUY")]
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] >= buy_to_cover_threshold))
                | ((df["Side"] == "Ask") & (df["Dif"] <= sell_short_threshold)) ]
    df.loc[df["Side"] == "Bid", "Multiplier"] = df.loc[df["Side"] == "Bid", "Dif"] - buy_to_cover_threshold
    df.loc[df["Side"] == "Ask", "Multiplier"] = df.loc[df["Side"] == "Ask", "Dif"] - sell_short_threshold
    df["VolumeSum"] = df["TotalSize"] * df["Multiplier"]
    df = df[["Time", "VolumeSum", "BUY", "SELL"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first"})
    df = df[(df["VolumeSum"] > size_threshold) | (df["VolumeSum"] < -size_threshold)].reset_index()
    df["Long"] = df["VolumeSum"] > size_threshold
    df["Short"] = df["VolumeSum"] < -size_threshold
    if len(df) < 2:
        return 0, None

    df["Sell Action"] = df["Short"] & df["Long"].shift()
    if df.iloc[0, df.columns.get_loc("Short")]:
        df.iloc[0, df.columns.get_loc("Sell Action")] = True
    df["Buy Action"] = df["Long"] & df["Short"].shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "BUY"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "SELL"])

    if df.iloc[len(df) - 1, df.columns.get_loc("Short")]:
        df.loc[len(df)] = [0, 0, 0, 0, 0, 0, 0, 0, -last_buy_to_cover]
    return df["Gain"].sum(), df

In [6]:
import numpy as np
import pandas as pd
def find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    df_dict = {}
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()]
        df_dict[date] = df

    long_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), len(dates) + 3))
    long_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    long_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    short_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), len(dates) + 3))
    short_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    short_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    for i in range(len(long_arr)):
        buy_threshold, sell_threshold = long_arr[i, 0], long_arr[i, 1]
        for j, date in enumerate(dates):
            long_arr[i, j + 2], _ = calc_gain_long(df_dict[date].copy(), buy_threshold, sell_threshold, size_threshold)
            short_arr[i, j + 2], _ = calc_gain_short(df_dict[date].copy(), buy_threshold, sell_threshold, size_threshold)
    long_arr[:, -1] = long_arr[:, 2:-1].sum(axis=1)
    long_df = pd.DataFrame(long_arr, columns = ["Buy Threshold", "Sell Threshold"]+dates+["Sum"])
    long_df = long_df.round(3)
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_thresholds_long.csv", index=False, date_format="%Y-%m-%d")
    long_df = long_df.sort_values(by=["Sum"], ascending=False)
    short_arr[:, -1] = short_arr[:, 2:-1].sum(axis=1)
    short_df = pd.DataFrame(short_arr, columns = ["Buy Threshold", "Sell Threshold"]+dates+["Sum"])
    short_df = short_df.round(3)
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_thresholds_short.csv", index=False, date_format="%Y-%m-%d")
    short_df = short_df.sort_values(by=["Sum"], ascending=False)
    return long_df.iloc[0].tolist(), short_df.iloc[0].tolist()

symbol = "TSLA"
buy_thresholds = (-np.arange(50) / 200) + .25
sell_thresholds = (-np.arange(50) / 200)
size_threshold = 0
#for size_threshold in range(1, 100, 1):
hour = 13
days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
dates = [f"2023-06-{day}" for day in days]
print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold)}")
# days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# for hour in range(14, 20):
#     print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold)}")


13: ([0.06, -0.215, -0.12, -2.69, 6.44, 3.48, 3.75, -6.53, 6.6, -4.12, 4.75, 4.8, 5.02, -1.42, 1.24, 21.2], [0.065, -0.13, -0.74, 0.0, -0.19, -0.28, 0.24, 0.68, -1.24, 1.22, -0.88, 2.62, 0.82, 0.04, -0.08, 2.21])


In [17]:
import numpy as np
import pandas as pd
from datetime import datetime
def update_thresholds(symbol, date, hour, buy_thresholds, sell_thresholds, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    long_df = pd.read_csv(f"tradestation_data/TSLA_{hour}_thresholds_long.csv")
    short_df = pd.read_csv(f"tradestation_data/TSLA_{hour}_thresholds_short.csv")
    cols = list(long_df.columns)
    print(cols)
    dates = cols[2:-1] + [date]
    print(dates)
    start = f'{date}T{hour}:00:00Z'
    end = f'{date}T{hour+1}:00:00Z'
    query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df["Dif"] = (df["Price"] - df["Last"])
    df = df.rename(columns={"CurTime": "Time"})
    query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
    df = pd.merge(df, df_3, on="Time")
    df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
    df = df[df["BUY"].notna() & df["SELL"].notna()]

    buy_arr = np.repeat(buy_thresholds, len(sell_thresholds))
    sell_arr = np.tile(sell_thresholds, len(buy_thresholds))
    long_arr = np.zeros(len(buy_thresholds) * len(sell_thresholds))
    short_arr = np.zeros(len(buy_thresholds) * len(sell_thresholds))
    for i in range(len(long_arr)):
        buy_threshold, sell_threshold = buy_arr[i], sell_arr[i]
        long_arr[i], _ = calc_gain_long(df.copy(), buy_threshold, sell_threshold, size_threshold)
        short_arr[i], _ = calc_gain_short(df.copy(), buy_threshold, sell_threshold, size_threshold)
    long_df[date] = long_arr
    long_df["Sum"] = long_df[dates].sum(axis=1)
    long_df = long_df[["Buy Threshold", "Sell Threshold"]+dates+["Sum"]]
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_thresholds_long.csv", index=False, date_format="%Y-%m-%d")
    long_df = long_df.sort_values(by=["Sum"], ascending=False)
    short_df[date] = long_arr
    short_df["Sum"] = short_df[dates].sum(axis=1)
    short_df = short_df[["Buy Threshold", "Sell Threshold"]+dates+["Sum"]]
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_thresholds_short.csv", index=False, date_format="%Y-%m-%d")
    short_df = short_df.sort_values(by=["Sum"], ascending=False)
    return long_df.iloc[0].tolist(), short_df.iloc[0].tolist()

symbol = "TSLA"
buy_thresholds = (-np.arange(50) / 200) + .25
sell_thresholds = (-np.arange(50) / 200)
size_threshold = 0
#for size_threshold in range(1, 100, 1):
hour = 13
date = datetime.utcnow().strftime("%Y-%m-%d")
print(f"{hour}: {update_thresholds(symbol, date, hour, buy_thresholds, sell_thresholds, size_threshold)}")
for hour in range(14, 20):
    print(f"{hour}: {update_thresholds(symbol, date, hour, buy_thresholds, sell_thresholds, size_threshold)}")


['Buy Threshold', 'Sell Threshold', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-26', 'Sum']
['2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-26', '2023-06-27']
13: ([0.045, -0.055, 1.089999999999975, -1.650000000000034, 6.440000000000055, 0.769999999999925, 3.430000000000007, -1.85000000000025, 3.9499999999999886, 1.2200000000000273, 3.5600000000000307, 2.8400000000000603, 19.799999999999784], [0.045, -0.055, 1.160000000000025, 0.9699999999999704, -0.0599999999999738, -2.2900000000000773, -0.3700000000000614, 5.319999999999766, -2.6499999999999773, 4.759999999999991, -2.130000000000024, 2.8400000000000603, 7.549999999999699])
['Buy Threshold', 'Sell Threshold', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-26', 'Sum']
['202

In [6]:
import numpy as np
import pandas as pd
def test_thresholds(symbol, dates, hour, buy_threshold, sell_threshold, buy_to_cover_threshold, sell_short_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    long_arr = np.empty((0,7))
    short_arr = np.empty((0,7))
    long_gains = []
    short_gains = []
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()]

        long_gain, long_df = calc_gain_long(df.copy(), buy_threshold, sell_threshold, size_threshold)
        long_gains.append(long_gain)
        if long_df is not None:
            long_df = long_df[["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"]]
            long_arr = np.concatenate((long_arr, long_df.to_numpy()), axis=0)
        short_gain, short_df = calc_gain_short(df.copy(), buy_to_cover_threshold, sell_short_threshold, size_threshold)
        short_gains.append(short_gain)
        if short_df is not None:
            short_df = short_df[["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"]]
            short_arr = np.concatenate((short_arr, short_df.to_numpy()), axis=0)
    long_df = pd.DataFrame(long_arr, columns=["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"])
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_test_long.csv")
    short_df = pd.DataFrame(short_arr, columns=["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"])
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_test_short.csv")
    return long_gains, short_gains
symbol = "TSLA"
buy_thresholds = {13: .045, 14: .115, 15: .08, 16: .055, 17: .033, 18: .025, 19: .02}
sell_thresholds = {13: -.055, 14: -.08, 15: -.185, 16: -.07, 17: -.09, 18: -.215, 19: -.065}
buy_to_cover_thresholds = {13: .045, 14: .24, 15: .175, 16: .085, 17: .035, 18: .08, 19: .02}
sell_short_thresholds = {13: -.055, 14: -.12, 15: -.115, 16: -.07, 17: -.09, 18: -.065, 19: -.065}
size_threshold = 0
hour = 13
days = ["13", "14", "15", "16", "20", "21", "22", "23", "26"]
dates = [f"2023-06-{day}" for day in days]
print(test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour], size_threshold))
days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26"]
dates = [f"2023-06-{day}" for day in days]
for hour in range(14, 20):
    print(test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour], size_threshold))


([1.089999999999975, -1.650000000000034, 6.440000000000055, 0.769999999999925, 3.430000000000007, -1.8500000000002501, 3.9499999999999886, 1.2200000000000273, 3.5600000000000307], [1.160000000000025, 0.9699999999999704, -0.05999999999997385, -2.2900000000000773, -0.3700000000000614, 5.319999999999766, -2.6499999999999773, 4.759999999999991, -2.130000000000024])
([0, 3.219999999999999, -1.2800000000000296, 0.2400000000000091, 0.3199999999999932, 0.5, 0.7599999999999909, 0.0, 1.5000000000000568, 0.0, -0.8300000000000125, 0.0], [0, 0, 2.3000000000000114, 0.6999999999999886, 0, 0.8000000000000114, 2.6999999999999886, 0, 1.169999999999959, 2.680000000000007, 0, 0])
([0.8299999999999841, 0.30000000000001137, 0, 1.9699999999999704, 0, 0, 0.5400000000000205, 0, 0.40999999999996817, 4.269999999999982, 2.4799999999999613, 0], [0, 3.670000000000016, 0, -2.180000000000007, 0, 0, 0, 0, 3.8899999999999864, 0, -2.0300000000000296, 0])
([-0.6999999999999886, 1.1399999999999864, 0.8600000000000136, 1.8

In [2]:
import numpy as np
import pandas as pd
spread = .1
def est_gain_long(df, buy_threshold, sell_threshold, size_threshold):
    last_sell = df.iloc[len(df) - 1, df.columns.get_loc("Bid")]
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] >= buy_threshold))
                | ((df["Side"] == "Ask") & (df["Dif"] <= sell_threshold))]
    df.loc[df["Side"] == "Bid", "Multiplier"] = df.loc[df["Side"] == "Bid", "Dif"] - buy_threshold
    df.loc[df["Side"] == "Ask", "Multiplier"] = df.loc[df["Side"] == "Ask", "Dif"] - sell_threshold
    df["VolumeSum"] = df["TotalSize"] * df["Multiplier"]
    df = df[["Time", "VolumeSum", "Ask", "Bid"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "Ask": "first", "Bid": "first"})
    df = df[(df["VolumeSum"] > size_threshold) | (df["VolumeSum"] < -size_threshold)].reset_index()
    df["Long"] = df["VolumeSum"] > size_threshold
    df["Short"] = df["VolumeSum"] < -size_threshold
    if len(df) < 2:
        return 0, None

    df["Ask"] += spread
    df["Bid"] -= spread

    df["Buy Action"] = df["Long"] & df["Short"].shift()
    if df.iloc[0, df.columns.get_loc("Long")]:
        df.iloc[0, df.columns.get_loc("Buy Action")] = True
    df["Sell Action"] = df["Short"] & df["Long"].shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "Ask"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "Bid"])

    if df.iloc[len(df) - 1, df.columns.get_loc("Long")]:
        df.loc[len(df)] = [0, 0, 0, 0, 0, 0, 0, 0, last_sell]
    return df["Gain"].sum(), df

def est_gain_short(df, buy_to_cover_threshold, sell_short_threshold, size_threshold):
    last_buy_to_cover = df.iloc[len(df) - 1, df.columns.get_loc("Ask")]
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] >= buy_to_cover_threshold))
                | ((df["Side"] == "Ask") & (df["Dif"] <= sell_short_threshold)) ]
    df.loc[df["Side"] == "Bid", "Multiplier"] = df.loc[df["Side"] == "Bid", "Dif"] - buy_to_cover_threshold
    df.loc[df["Side"] == "Ask", "Multiplier"] = df.loc[df["Side"] == "Ask", "Dif"] - sell_short_threshold
    df["VolumeSum"] = df["TotalSize"] * df["Multiplier"]
    df = df[["Time", "VolumeSum", "Ask", "Bid"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "Ask": "first", "Bid": "first"})
    df = df[(df["VolumeSum"] > size_threshold) | (df["VolumeSum"] < -size_threshold)].reset_index()
    df["Long"] = df["VolumeSum"] > size_threshold
    df["Short"] = df["VolumeSum"] < -size_threshold
    if len(df) < 2:
        return 0, None

    df["Ask"] += spread
    df["Bid"] -= spread
    
    df["Sell Action"] = df["Short"] & df["Long"].shift()
    if df.iloc[0, df.columns.get_loc("Short")]:
        df.iloc[0, df.columns.get_loc("Sell Action")] = True
    df["Buy Action"] = df["Long"] & df["Short"].shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "Ask"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "Bid"])

    if df.iloc[len(df) - 1, df.columns.get_loc("Short")]:
        df.loc[len(df)] = [0, 0, 0, 0, 0, 0, 0, 0, -last_buy_to_cover]
    return df["Gain"].sum(), df

In [3]:
import numpy as np
import pandas as pd
def find_est_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    df_dict = {}
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        df = df[["Time", "Side", "TotalSize", "Dif", "Ask", "Bid"]]
        df = df[df["Ask"].notna() & df["Bid"].notna()]
        df_dict[date] = df

    long_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), len(dates) + 3))
    long_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    long_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    short_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), len(dates) + 3))
    short_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    short_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    for i in range(len(long_arr)):
        buy_threshold, sell_threshold = long_arr[i, 0], long_arr[i, 1]
        for j, date in enumerate(dates):
            long_arr[i, j + 2], _ = est_gain_long(df_dict[date].copy(), buy_threshold, sell_threshold, size_threshold)
            short_arr[i, j + 2], _ = est_gain_short(df_dict[date].copy(), buy_threshold, sell_threshold, size_threshold)
    long_arr[:, -1] = long_arr[:, 2:-1].sum(axis=1)
    long_df = pd.DataFrame(long_arr, columns = ["Buy Threshold", "Sell Threshold"]+dates+["Sum"])
    long_df = long_df.round(3)
    long_df = long_df.sort_values(by=["Sum"], ascending=False)
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_est_thresholds_long.csv")
    short_arr[:, -1] = short_arr[:, 2:-1].sum(axis=1)
    short_df = pd.DataFrame(short_arr, columns = ["Buy Threshold", "Sell Threshold"]+dates+["Sum"])
    short_df = short_df.round(3)
    short_df = short_df.sort_values(by=["Sum"], ascending=False)
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_est_thresholds_short.csv")
    return long_df.iloc[0].tolist(), short_df.iloc[0].tolist()

symbol = "MSFT"
buy_thresholds = (-np.arange(50) / 100) + .25
sell_thresholds = (-np.arange(50) / 100) + .25
size_threshold = 0
#for size_threshold in range(1, 100, 1):
hour = 13
days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
dates = [f"2023-06-{day}" for day in days]
print(f"{hour}: {find_est_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold)}")
days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
dates = [f"2023-06-{day}" for day in days]
for hour in range(14, 20):
    print(f"{hour}: {find_est_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold)}")


13: ([0.13, -0.22, -0.91, 1.72, 3.93, -4.96, -0.6, -1.8, 2.41, 0.43, 1.9, 1.99, 0.59, -1.07, 1.83, 5.46], [0.14, -0.22, 2.67, 0.0, -0.51, -0.77, -0.6, -0.8, 0.12, -0.05, 0.97, 0.0, 0.0, -0.38, -1.45, -0.8])
14: ([0.09, -0.22, 0.0, 0.0, 0.0, 0.0, 0.0, 1.96, 0.43, 0.0, 0.0, 0.0, 1.18, 0.0, 0.0, 0.0, 0.97, 0.0, 4.54], [0.06, -0.1, 0.0, 0.0, 0.0, 0.06, 0.0, -0.25, -0.09, 0.0, 0.0, 1.38, -0.18, 0.0, 0.0, 0.0, 0.8, 0.0, 1.72])
15: ([0.06, -0.12, -0.11, 0.0, 0.0, 0.0, 0.0, -0.02, 1.85, 0.0, -0.62, 0.22, 0.17, -2.66, 2.62, -0.41, 0.0, 0.0, 1.04], [0.05, -0.08, 0.0, 0.57, 0.0, 0.71, 0.0, -0.12, 1.3, -0.11, -0.02, 0.0, 0.21, 0.17, 0.0, 0.0, 0.04, 0.67, 3.42])
16: ([0.03, -0.17, -1.38, 0.74, -0.54, 1.94, 0.16, -0.0, -0.42, 2.56, 1.4, -0.06, 1.09, -1.15, 0.57, -1.93, -0.89, 0.94, 3.03], [0.05, -0.24, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3])
17: ([0.03, -0.1, -0.78, 1.43, 1.34, 0.09, -0.34, 1.23, -0.12, -0.61, -0.05, 1.39, -0.73, -0.45, 1.73, 0.23, -0.51

In [22]:
import numpy as np
import pandas as pd
def est_test_thresholds(symbol, dates, hour, buy_threshold, sell_threshold, buy_to_cover_threshold, sell_short_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    long_arr = np.empty((0,7))
    short_arr = np.empty((0,7))
    long_gains = []
    short_gains = []
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        df = df[["Time", "Side", "TotalSize", "Dif", "Ask", "Bid"]]
        df = df[df["Ask"].notna() & df["Bid"].notna()]

        long_gain, long_df = est_gain_long(df.copy(), buy_threshold, sell_threshold, size_threshold)
        long_gains.append(long_gain)
        if long_df is not None:
            long_df = long_df[["Time", "VolumeSum", "Ask", "Bid", "Buy Action", "Sell Action", "Gain"]]
            long_arr = np.concatenate((long_arr, long_df.to_numpy()), axis=0)
        short_gain, short_df = est_gain_short(df.copy(), buy_to_cover_threshold, sell_short_threshold, size_threshold)
        short_gains.append(short_gain)
        if short_df is not None:
            short_df = short_df[["Time", "VolumeSum", "Ask", "Bid", "Buy Action", "Sell Action", "Gain"]]
            short_arr = np.concatenate((short_arr, short_df.to_numpy()), axis=0)
    long_df = pd.DataFrame(long_arr, columns=["Time", "VolumeSum", "Ask", "Bid", "Buy Action", "Sell Action", "Gain"])
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_est_test_long.csv")
    short_df = pd.DataFrame(short_arr, columns=["Time", "VolumeSum", "Ask", "Bid", "Buy Action", "Sell Action", "Gain"])
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_est_test_short.csv")
    return long_gains, short_gains
symbol = "TSLA"
buy_thresholds = {13: .0, 14: .115, 15: .08, 16: .055, 17: .033, 18: .025, 19: .02}
sell_thresholds = {13: -.09, 14: -.08, 15: -.185, 16: -.07, 17: -.09, 18: -.215, 19: -.065}
buy_to_cover_thresholds = {13: .045, 14: .24, 15: .175, 16: .085, 17: .035, 18: .08, 19: .02}
sell_short_thresholds = {13: -.055, 14: -.12, 15: -.115, 16: -.07, 17: -.09, 18: -.065, 19: -.065}
size_threshold = 0
hour = 13
days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
dates = [f"2023-06-{day}" for day in days]
print(est_test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour], size_threshold))
# days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# for hour in range(14, 20):
#     print(est_test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour], size_threshold))


([-1.8199999999999363, -3.5499999999998977, 7.150000000000034, 5.520000000000209, 3.480000000000075, -4.970000000000255, 7.099999999999994, -2.0199999999998965, 2.3899999999999295, 7.6099999999998715, 1.4500000000001023, -1.759999999999934, 1.4899999999999523], [1.0000000000002842, 2.639999999999702, -1.2599999999998204, -1.7900000000003615, -0.03000000000020009, 5.709999999999809, 3.0800000000000978, 6.369999999999891, -1.4699999999999989, 1.820000000000249, -4.169999999999845, 0.009999999999877218, -0.3200000000002774])


In [14]:
def get_max_volume(symbol, start_date, end_date):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    start = f'2023-06-{start_date}T13:30:00Z'
    end = f'2023-06-{end_date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    return df["TotalSize"].max(), df["TotalSize"].min(), df["TotalSize"].mean(), df["Dif"].max(), df["Dif"].min(), df["Dif"].mean() 

symbol = "TSLA"
start_date = "08"
end_date = "14"
print(get_max_volume(symbol, start_date, end_date))

(225600.0, 1.0, 487.7769264200571, 5.219999999999999, -250.45, -3.205123899786772)


In [6]:
import pandas as pd
import numpy as np
def get_spread(symbol, start_date, end_date, threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    start = f'2023-06-{start_date}T13:30:00Z'
    end = f'2023-06-{end_date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    query = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query)))
    df_3 = df_3.rename(columns={"Time": "CurTime"})
    df = pd.merge(df_1, df_2, on="CurTime")
    #print(df)
    df = pd.merge(df, df_3, on="CurTime")
    #print(df)
    df = df[df["BUY"].notnull() & df["SELL"].notnull() & df["Last"].notnull()]
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df = df.loc[df["Dif"] >= threshold]
    df["BUY"] = pd.to_numeric(df["BUY"])
    df["SELL"] = pd.to_numeric(df["SELL"])
    df["Buy Dif"] = df["BUY"] - df["Last"]
    df["Sell Dif"] = df["Last"] - df["SELL"]
    df["Spread"] = df["BUY"] - df["SELL"]
    #print(df["BUY"].min())
    #print(df["Last"].min())
    return df["Buy Dif"].max(), df["Buy Dif"].min(), df["Buy Dif"].mean(), df["Sell Dif"].max(), df["Sell Dif"].min(), df["Sell Dif"].mean(), df["Spread"].max(), df["Spread"].min(), df["Spread"].mean()

symbol = "TSLA"
start_date = "13"
end_date = "14"
for threshold in np.arange(0, .1, .01):
    print(get_spread(symbol, start_date, end_date, threshold))

(0.42470000000002983, -0.5999999999999943, 0.011979456415278771, 0.7199999999999989, -0.3147000000000162, 0.02557686092066695, 0.20000000000001705, -0.03999999999996362, 0.03755631733594572)
(0.42470000000002983, -0.5999999999999943, 0.008795021337126343, 0.7199999999999989, -0.3147000000000162, 0.03182612612612713, 0.20000000000001705, -0.03999999999996362, 0.040621147463253474)
(0.2950000000000159, -0.29999999999998295, 0.005403648648648475, 0.3300000000000409, -0.26499999999998636, 0.03718418918919046, 0.20000000000001705, -0.03000000000002956, 0.042587837837838934)
(0.2810000000000059, -0.29999999999998295, 0.0026456959706953843, 0.3300000000000409, -0.2300000000000182, 0.041447710622712255, 0.20000000000001705, -0.03000000000002956, 0.04409340659340764)
(0.2810000000000059, -0.29999999999998295, -0.0033564753004013065, 0.3300000000000409, -0.2300000000000182, 0.04944459279038953, 0.20000000000001705, -0.009999999999990905, 0.04608811748998822)
(0.2810000000000059, -0.2999999999999

In [5]:
import pandas as pd
# symbol = "TSLA"
# days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "06", "07", "10", "11", "12", "13", "14", "17", "18", "19", "20", "21"]
# dates = dates + [f"2023-07-{day}" for day in days]
symbols = ['AMZN', 'AAPL', 'GOOG']
days = ["19", "20", "21"]
dates = [f"2023-07-{day}" for day in days]
for symbol in symbols:
    hour = 13
    balance = 0
    ret = []
    for day in dates:
        start = f"{day}T{hour}:30:00Z"
        end = f"{day}T{hour+1}:00:00Z"
        collection = f"{symbol}_10sec_ts_lvl1"
        start_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": start})))["Last"].loc[0]
        end_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": end})))["Last"].loc[0]
        dif = (end_price - start_price).round(3)
        balance += dif
        ret.append(dif)
    ret.append(balance.round(3))
    print(f"{hour}: {ret}")
    for hour in range(14, 20):
        balance = 0
        ret = []
        for day in dates:
            start = f"{day}T{hour}:00:00Z"
            end = f"{day}T{hour+1}:00:00Z"
            collection = f"{symbol}_10sec_ts_lvl1"
            start_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": start})))["Last"].loc[0]
            end_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": end})))["Last"].loc[0]
            dif = (end_price - start_price).round(3)
            balance += dif
            ret.append(dif)
        ret.append(balance.round(3))
        print(f"{hour}: {ret}")
    print(f"Total Balance: {balance}\n")
    balance = 0
    ret = []
    for day in dates:
        start = f"{day}T13:30:00Z"
        end = f"{day}T20:00:00Z"
        collection = f"{symbol}_10sec_ts_lvl1"
        start_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": start})))["Last"].loc[0]
        end_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": end})))["Last"].loc[0]
        dif = (end_price - start_price).round(3)
        balance += dif
        ret.append(dif)
    ret.append(balance.round(3))
    print(f"Daily: {ret}")
    start = f"{dates[0]}T13:30:00Z"
    end = f"{dates[-1]}T20:00:00Z"
    collection = f"{symbol}_10sec_ts_lvl1"
    start_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": start})))["Last"].loc[0]
    end_price = pd.DataFrame(list(mongo.mongoDB[collection].find({"CurTime": end})))["Last"].loc[0]     
    print(f"Overall: {(end_price - start_price).round(3)}")

13: [1.37, -0.1, -1.39, -0.12]
14: [-0.115, -1.628, -0.28, -2.023]
15: [0.575, -0.272, 0.65, 0.953]
16: [-0.015, -0.56, -0.345, -0.92]
17: [0.145, -0.0, 0.595, 0.74]
18: [0.01, -1.135, -0.365, -1.49]
19: [0.06, -0.355, -0.155, -0.45]
Total Balance: -0.44999999999999996

Daily: [2.03, -4.05, -1.29, -3.31]
Overall: -3.45
13: [-0.3, 0.74, -0.22, 0.22]
14: [0.15, -1.315, -0.64, -1.805]
15: [0.42, 0.515, -0.055, 0.88]
16: [1.69, -0.442, -0.932, 0.316]
17: [-0.27, -0.178, 0.937, 0.489]
18: [0.75, -1.04, -0.52, -0.81]
19: [-0.44, -0.14, -0.65, -1.23]
Total Balance: -1.23

Daily: [2.0, -1.86, -2.08, -1.94]
Overall: -1.17
13: [-0.2, 1.47, -0.64, 0.63]
14: [-0.57, -1.972, -0.03, -2.572]
15: [0.26, -0.248, 0.179, 0.191]
16: [-1.17, -0.755, -0.449, -2.374]
17: [-0.18, -0.125, 0.535, 0.23]
18: [0.11, -1.17, -0.22, -1.28]
19: [-0.39, 0.29, 0.135, 0.035]
Total Balance: 0.034999999999999976

Daily: [-2.14, -2.51, -0.49, -5.14]
Overall: -4.56
